In [1]:
from bg_atlasapi import show_atlases
show_atlases()
from bg_atlasapi.bg_atlas import BrainGlobeAtlas
import numpy as np
import os
import json
import pandas as pd
from pathlib import Path
import shutil
import bpy, bmesh
import os

╭───────────────────────────────── Brainglobe Atlases ─────────────────────────────────╮
│                                                                           Latest     │
│  Name                              Downloaded  Updated  Local version    version     │
│  allen_mouse_25um                      ✔          ✔          1.2           1.2       │
│  osten_mouse_100um                     ✔          ✔          1.1           1.1       │
│  whs_sd_rat_39um                       ✔          ✔          1.1           1.1       │
│  example_mouse_100um                                                       1.2       │
│  allen_mouse_10um                                                          1.2       │
│  allen_mouse_50um                                                          1.2       │
│  allen_mouse_100um                                                         1.2       │
│  mpin_zfish_1um                                                            1.0       │
│  allen_human_500um                                                         0.1       │
│  kim_mouse_10um                                                            1.0       │
│  kim_mouse_25um                                                            1.0       │
│  kim_mouse_50um                                                            1.0       │
│  kim_mouse_100um                                                           1.0       │
│  osten_mouse_10um                                                          1.1       │
│  osten_mouse_25um                                                          1.1       │
│  osten_mouse_50um                                                          1.1       │
│  allen_cord_20um                                                           1.0       │
│  azba_zfish_4um                                                            1.1       │
│  perens_lsfm_mouse_20um                                                    1.0       │
│  admba_3d_e11_5_mouse_16um                                                 1.0       │
│  admba_3d_e13_5_mouse_16um                                                 1.0       │
│  admba_3d_e15_5_mouse_16um                                                 1.0       │
│  admba_3d_e18_5_mouse_16um                                                 1.0       │
│  admba_3d_p4_mouse_16.752um                                                1.0       │
│  admba_3d_p14_mouse_16.752um                                               1.0       │
│  admba_3d_p28_mouse_16.752um                                               1.0       │
│  admba_3d_p56_mouse_25um                                                   1.0       │
│  princeton_mouse_20um                                                      1.0       │
│  kim_dev_mouse_stp_10um                                                    1.1       │
│  kim_dev_mouse_idisco_10um                                                 1.1       │
│  kim_dev_mouse_mri_a0_10um                                                 1.1       │
│  kim_dev_mouse_mri_adc_10um                                                1.1       │
│  kim_dev_mouse_mri_dwi_10um                                                1.1       │
│  kim_dev_mouse_mri_fa_10um                                                 1.1       │
│  kim_dev_mouse_mri_mtr_10um                                                1.1       │
│  kim_dev_mouse_mri_t2_10um                                                 1.1       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [2]:
def intermediate_meta(atlas_name):
    """Build the metadata file

    Parameters
    ----------
    atlas_name : string
        bg-atlas name
    """
    meta_file = os.path.join(PATH,atlas_name,"meta.json")

    with open(meta_file, 'w', encoding='utf-8') as f:
        json.dump(atlas.metadata, f, ensure_ascii=False, indent=4)

def intermediate_ref_image(atlas_name):
    """Build the intermediate reference image. Input is the bg-atlas reference image (uint16) output is a flattened .bytes file.

    Parameters
    ----------
    atlas_name : string
        bg-atlas name
    """
    reference_file = os.path.join(PATH,atlas_name,"reference.bytes")

    atlas = BrainGlobeAtlas(atlas_name)
    reference = atlas.reference

    if not isinstance(reference[0,0,0], np.uint16):
        print("Warning: atlas has incorrect reference image format")

    reference.flatten().tofile(reference_file)

def intermediate_annot_image(atlas_name):
    """Build the intermediate annotation image. Input is the bg-atlas annotation image (uint32) output is a flattened .bytes file

    Parameters
    ----------
    atlas_name : string
        bg-atlas name
    """
    annotation_file = os.path.join(PATH,atlas_name,"annotation.bytes")

    atlas = BrainGlobeAtlas(atlas_name)
    annotation = atlas.annotation

    if not isinstance(annotation[0,0,0], np.uintc):
        print("Warning: atlas has incorrect reference image format")

    annotation.flatten().tofile(annotation_file)

def intermediate_mesh_centers(atlas_name):
    """Build the intermediate mesh center CSV files"""
    mesh_center_file = os.path.join(PATH, atlas_name, "mesh_centers.csv")

    atlas = BrainGlobeAtlas(atlas_name)
    root_name = "root"

    all_structures = atlas.get_structure_descendants(root_name)
    all_structures.insert(0,root_name)

    df = pd.DataFrame(columns=["structure_name","ap","ml","dv","ap_um","ml_um","dv_um"])

    res = atlas.metadata["resolution"]

    for i, structure in enumerate(all_structures):
        mask = atlas.get_structure_mask(atlas.structures[structure]["id"])
        if not np.any(mask.flatten()):
            df.loc[i] = [structure, -1, -1, -1, -1, -1, -1]
        else:
            coords = np.mean(np.argwhere(mask), axis=0)
            df.loc[i] = [structure, coords[0], coords[1], coords[2], coords[0]*res[0], coords[1]*res[1], coords[2]*res[2]]

    df.to_csv(mesh_center_file, float_format='%0.2f')

def intermediate_structures(atlas_name):
    """Save the structure hierarchy data (this is a symlink to the brainglobe file)

    Parameters
    ----------
    atlas_name : string
        bg-atlas name
    """
    atlas = BrainGlobeAtlas(atlas_name)

    shutil.copyfile(os.path.join(atlas.root_dir, 'structures.json'),os.path.join(PATH,atlas_name,"structures.json"))

# Function to recalculate normals and save the object
def recalculate_normals_and_save(obj_file):
    # Import the .obj file
    bpy.ops.import_scene.obj(filepath=obj_file)

    # Select the imported object
    obj = bpy.context.selected_objects[0]

    # Switch to Edit Mode and recalculate normals
    bpy.context.view_layer.objects.active = obj
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.mesh.normals_make_consistent(inside=False)
    bpy.ops.object.mode_set(mode='OBJECT')

    # Save the object with recalculated normals
    bpy.ops.export_scene.obj(filepath=obj_file, use_selection=True, use_normals=True, use_materials=False)

    # Unlink the imported object (remove it from the scene)
    bpy.data.objects.remove(obj)

def cleanup_folder(atlas_name):
    """Remove extra #L.obj #LL.obj and *.mat files

    Parameters
    ----------
    atlas_name : _type_
        _description_
    """
    def is_desired_filename(filename):
        return filename.endswith(".obj") and filename[0].isdigit() and not filename.endswith("L.obj") and not filename.endswith("LL.obj")

    atlas = BrainGlobeAtlas(atlas_name)

    folder = os.path.join(atlas.root_dir,'meshes')

    # Get a list of all files in the folder
    all_files = os.listdir(folder)

    # Iterate through the files and delete those that don't match the pattern
    for filename in all_files:
        if not is_desired_filename(filename):
            file_to_delete = os.path.join(folder, filename)
            os.remove(file_to_delete)

def repair_normals(atlas_name):
    """Load all OBJ files and recalculate the normals

    Parameters
    ----------
    atlas_name : _type_
        _description_
    """
    atlas = BrainGlobeAtlas(atlas_name)

    folder = os.path.join(atlas.root_dir,'meshes')

    # List all .obj files in the folder
    obj_files = [f for f in os.listdir(folder) if f.endswith(".obj") and not f.endswith("L.obj") and not f.endswith("LL.obj")]

    # Process each .obj file
    for obj_file in obj_files:
        recalculate_normals_and_save(os.path.join(folder, obj_file))


def intermediate_mesh_files(atlas_name):
    """Run the Blender slicer to make the single-hemisphere files
    """
    atlas = BrainGlobeAtlas(atlas_name)


    slice_depth = - atlas.metadata['shape'][2] * atlas.metadata['resolution'][2] / 2

    folder = os.path.join(atlas.root_dir,'meshes')

    # write a text file with the location of the files
    mesh_path_file = os.path.join(PATH,atlas_name,"mesh_path.txt")

    with open(mesh_path_file, 'w') as f:
        f.write(folder)
        f.close()

    files = [file for file in os.listdir(folder) if file.endswith('.obj')]

    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete()

    for file in files:
        fpath = os.path.join(folder, file)
        fpath_out = os.path.join(folder, os.path.splitext(file)[0] + 'L' + os.path.splitext(file)[1])

        if os.path.exists(fpath_out):
            continue

        fpath_mtl = os.path.join(folder, os.path.splitext(file)[0] + 'L.mtl')

        bpy.ops.object.select_all(action='DESELECT')
        bpy.ops.import_scene.obj(filepath=fpath)
        bpy.context.view_layer.objects.active = bpy.context.selected_objects[0]

        #obj = bpy.context.object
    #    bpy.ops.transform.resize(value=(0.001, 0.001, 0.001))

        bpy.ops.object.editmode_toggle()
        bpy.ops.mesh.select_all(action='SELECT')
        bpy.ops.mesh.bisect(plane_co=(0.0,slice_depth,0.0),plane_no=(0.0,-1.0,0.0),use_fill=True,clear_outer=True)
        bpy.ops.object.modifier_add(type='TRIANGULATE')
        bpy.ops.object.editmode_toggle()

        bpy.ops.export_scene.obj(filepath=fpath_out)

        bpy.ops.object.delete()

        # trash the .mtl file
        if os.path.exists(fpath_mtl):
            os.remove(fpath_mtl)

In [3]:
# get all the root mesh files for the atlases we have downloaded
PATH = "./data"

downloaded = ["allen_mouse_25um", "osten_mouse_100um","whs_sd_rat_39um"]
atlases = []

for atlas_name in downloaded:
    atlas = BrainGlobeAtlas(atlas_name, check_latest=True)
    atlases.append(atlas)

    if not os.path.isdir(os.path.join(PATH,atlas_name)):
        os.mkdir(os.path.join(PATH,atlas_name))
        
    # intermediate_meta(atlas_name)
    # intermediate_structures(atlas_name)
    # intermediate_ref_image(atlas_name)
    # intermediate_annot_image(atlas_name)
    # intermediate_mesh_centers(atlas_name)
    cleanup_folder(atlas_name)
    repair_normals(atlas_name)
    intermediate_mesh_files(atlas_name)

(  0.0000 sec |   0.0000 sec) Importing OBJ 'C:\\Users\\Dan\\.brainglobe\\allen_mouse_25um_v1.2\\meshes\\1.obj'...
  (  0.0000 sec |   0.0000 sec) Parsing OBJ file...
    (  0.0030 sec |   0.0030 sec) Done, loading materials and images...
	Material not found MTL: 'C:\\Users\\Dan\\.brainglobe\\allen_mouse_25um_v1.2\\meshes\\1.mtl'
    (  0.0030 sec |   0.0030 sec) Done, building geometries (verts:229 faces:450 materials: 1 smoothgroups:1) ...
    (  0.0080 sec |   0.0080 sec) Done.
  (  0.0080 sec |   0.0080 sec) Finished importing: 'C:\\Users\\Dan\\.brainglobe\\allen_mouse_25um_v1.2\\meshes\\1.obj'
Progress: 100.00%

    (  0.0000 sec |   0.0000 sec) OBJ Export path: 'C:\\Users\\Dan\\.brainglobe\\allen_mouse_25um_v1.2\\meshes\\1.obj'
          (  0.0058 sec |   0.0058 sec) Finished writing geometry of '1.001'.
      (  0.0058 sec |   0.0058 sec) Finished exporting geometry, now exporting materials
      (  0.0058 sec |   0.0058 sec) OBJ Export Finished
Progress: 100.00%

(  0.0000 sec 

KeyboardInterrupt: 

KeyboardInterrupt: 

Error: Python: 
Location: c:\proj\VBL\BrainAtlas\Pipelines\ba-venv\lib\site-packages\bpy\3.6\scripts\modules\bpy\ops.py:113


RuntimeError: Error: Python: 
Location: c:\proj\VBL\BrainAtlas\Pipelines\ba-venv\lib\site-packages\bpy\3.6\scripts\modules\bpy\ops.py:113
